In [2]:
import tqdm as notebook_tqdm
from langchain_google_genai import GoogleGenerativeAI

C:\Users\pragy\anaconda3\envs\llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your api key here")

api_key = os.environ["GOOGLE_API_KEY"]

Provide your api key here ········


In [4]:
llm = GoogleGenerativeAI(model="models/text-bison-001", google_api_key=api_key, temperature=0.7)
print(llm.invoke("suggest some names for a tech startup"))

* **Aardvark** - A tech startup that helps people find information and answers.
* **Blip** - A tech startup that creates short-form video content.
* **Carousel** - A tech startup that creates social media platforms.
* **Digicube** - A tech startup that creates virtual reality experiences.
* **Ember** - A tech startup that creates software for businesses.
* **Flo** - A tech startup that creates mobile apps for women.
* **Glitch** - A tech startup that creates gaming platforms.
* **Hubble** - A tech startup that creates space exploration technology.
* **Jukebox** - A tech startup that creates music streaming platforms.
* **Kaleidoscope** - A tech startup that creates augmented reality experiences.
* **Launchpad** - A tech startup that helps people launch their own businesses.
* **Mosaic** - A tech startup that creates artificial intelligence solutions.
* **Narwhal** - A tech startup that creates cryptocurrency platforms.
* **Oculus** - A tech startup that creates virtual reality headsets

In [6]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.output_parsers import StrOutputParser

pt_name1 = PromptTemplate(
    input_variables = ['domain'],
    template = "give A SINGLE name for a {domain} startup"
)

#pt_name.format(domain="healthcare")

pt_name2 = PromptTemplate(
    input = ['name'],
    template = "suggest some services that {name} provides"
)

op = StrOutputParser()

chain1 = LLMChain(prompt = pt_name1, llm = llm, output_parser = op, output_key = 'name')
chain2 = LLMChain(prompt = pt_name2, llm = llm, output_parser = op, output_key = 'services')

result = chain1.invoke({'domain':'healthcare'})

if isinstance(result, dict):
    print(result['name'])

C:\Users\pragy\AppData\Local\Temp\ipykernel_18688\3678337909.py:19: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  chain1 = LLMChain(prompt = pt_name1, llm = llm, output_parser = op, output_key = 'name')


Wellbeing Clinic


# Simple Sequential Chains

In [7]:
from langchain.chains import SimpleSequentialChain

si_sq_chain = SimpleSequentialChain(chains = [chain1, chain2])
response = si_sq_chain.invoke('healthcare')

if isinstance(response, dict):
    print(f"Startup name : {result['name']}\n")
    print(response['output'])

Startup name : Wellbeing Clinic

**Healthie provides a variety of services to help people manage their health, including:**

* **A personal health record (PHR)** that allows users to store their medical information in one place, including prescriptions, test results, and immunization records.
* **A secure messaging platform** that allows users to communicate with their doctors and other healthcare providers.
* **A symptom checker** that helps users identify potential health problems and find resources for treatment.
* **A medication reminder** that helps users stay on top of their medication schedules.
* **A health coach** that provides personalized support and guidance to help users reach their health goals.

Healthie is an easy-to-use and convenient way for people to manage their health. With its wide range of services, Healthie can help people improve their health and well-being.


# Sequential Chains

In [10]:
from langchain.chains import SequentialChain

sq_chain = SequentialChain(
    chains = [chain1, chain2],
    input_variables = ['domain'],
    output_variables = ['name','services']
)

sq_chain.invoke('drones')

{'domain': 'drones',
 'name': 'AirGo Drones',
 'services': '* Aerial photography and videography\n* 3D mapping and modeling\n* Inspections of infrastructure, buildings, and other structures\n* Search and rescue operations\n* Agriculture and forestry applications\n* Law enforcement and security\n* Disaster relief\n* Media and entertainment\n* Other commercial and industrial applications'}

# Agents

In [15]:
from langchain.agents import AgentType, initialize_agent, load_tools

tools = load_tools(['wikipedia','llm-math'],llm=llm)
agent = initialize_agent(tools,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

agent.invoke("current gdp of india")



> Entering new AgentExecutor chain...
I need to find the current gdp of india
Action: wikipedia
Action Input: india gdp
Observation: Page: Economy of India
Summary: The economy of India is a developing mixed economy with a notable public sector in strategic sectors. It is the world's fifth-largest economy by nominal GDP and the third-largest by purchasing power parity (PPP); on a per capita income basis, India ranked 136th by GDP (nominal) and 125th by GDP (PPP). From independence in 1947 until 1991, successive governments followed the Soviet model and promoted protectionist economic policies, with extensive Sovietization, state intervention, demand-side economics, natural resources, bureaucrat driven enterprises and economic regulation. This is characterised as dirigism, in the form of the Licence Raj. The end of the Cold War and an acute balance of payments crisis in 1991 led to the adoption of a broad economic liberalisation in India and indicative planning. Since the start of the

{'input': 'current gdp of india', 'output': '35.436 trillion rupees'}

In [19]:
from secret_keys import serp_api
os.environ["SERPAPI_API_KEY"] = serp_api

tools = load_tools(['serpapi'], llm = llm)
agent = initialize_agent(tools,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose = True)
agent.invoke('logistic regression formula')



> Entering new AgentExecutor chain...
I need to search for the logistic regression formula
Action: Search
Action Input: logistic regression formula
Observation: ['The parameters of a logistic regression are most commonly estimated by maximum-likelihood estimation (MLE). This does not have a closed-form expression, unlike ...', 'The notation used for the test statistic is typically G2 G 2 = deviance (reduced) – deviance (full). This test procedure is analagous to the general linear F ...', 'Logistic regression uses a logistic function called a sigmoid function to map predictions and their probabilities. The sigmoid function refers ...', 'In logistic regression, the Wald test is calculated in the same manner. The formula for the Wald statistic is z j = b j s b j where s b j is an estimate of the ...', 'logit(P) = a + bX,. Which is assumed to be linear, that is, the log odds (logit) is assumed to be linearly related to X, our IV. So ...', "Logistic regression estimates the probability o

{'input': 'logistic regression formula',
 'output': 'logit(p) = log(p/(1-p))= (β0 + β1) + (β2 + β3 )*math'}

# Memory

In [28]:
from langchain.memory import ConversationBufferMemory

shop_template = PromptTemplate(
    input = ['category'],
    template = "suggest ONE shop name based on {category}"
)

memory = ConversationBufferMemory()
shop_chain = LLMChain(prompt=shop_template, llm = llm, output_parser = op, output_key = 'shop_name', memory=memory)
print(shop_chain.invoke('electronics')['shop_name'])
print(shop_chain.invoke('beauty')['shop_name'])


"Gadgets & Gizmos"
"The Beauty Bar"


In [29]:
print(shop_chain.memory.buffer)

Human: electronics
AI: "Gadgets & Gizmos"
Human: beauty
AI: "The Beauty Bar"


In [50]:
from langchain.chains import ConversationChain

llm = GoogleGenerativeAI(model="models/text-bison-001", google_api_key=api_key, temperature=0.7)
conv = ConversationChain(llm=llm)
print(conv.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [51]:
conv.run("what is gdp in one sentence")

'Gross domestic product (GDP) is a measure of the value of all goods and services produced within a country or region in a given period of time.'

In [52]:
conv.run("whats the gdp of india in one sentence")

"India's nominal GDP was $2.94 trillion in 2023."

In [53]:
conv.run("why is gdp used in one sentence")

"GDP is used to measure a country's economic output and to compare different countries' economies."

In [54]:
print(conv.memory.buffer)

Human: what is gdp in one sentence
AI: Gross domestic product (GDP) is a measure of the value of all goods and services produced within a country or region in a given period of time.
Human: whats the gdp of india in one sentence
AI: India's nominal GDP was $2.94 trillion in 2023.
Human: why is gdp used in one sentence
AI: GDP is used to measure a country's economic output and to compare different countries' economies.


In [55]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain

memory = ConversationBufferWindowMemory(k=1)
conv = ConversationChain(llm = llm, memory = memory)

In [57]:
conv.run('what is meant by equity in 1 sentence')

"Equity is the value of the owner's interest in a company."

In [58]:
conv.run('uses of equity in 1 sentence')

"Equity can be used to finance a company's operations, pay dividends to shareholders, or buy back shares."

In [59]:
conv.run('examples of equity in 1 sentence')

'Examples of equity include common stock, preferred stock, and retained earnings.'

In [61]:
print(conv.memory.buffer)

Human: examples of equity in 1 sentence
AI: Examples of equity include common stock, preferred stock, and retained earnings.
